In [29]:
import sys
import os
from pathlib import Path
import torch
import NEAT_LNC_node as neat
import pprint
from utils import *
from configuration import *
from NEAT_args import *

args = parser.parse_args([])
EA_args = EA_parser.parse_args([])

In [32]:
import re

results = torch.zeros([13,10,3])

args.DEVICE = 'cpu'

for ds in range(13):
    args.DATASET = ds
    args = FormulateArgs(args)
    valid_loader, datainfo = GetDataLoader(args, 'valid', path='../dataset/')
    test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset/')
    pprint.pprint(datainfo)
    
    for seed in range(10):
        args.SEED = seed
        for e, eps_train in enumerate([0.0, 0.05, 0.1]):
            args.e_train = eps_train

            logname = f"NEAT_data_{datainfo['dataname']}_seed_{args.SEED:02d}_epsilon_{eps_train}.log"
            logpath = f'./log/{logname}'
            
            if os.path.exists(logpath):

                with open(logpath, 'r', encoding='utf-8') as file:
                    data = file.read()

                epoch_pattern = r"\|\s+(\d+)\s+\|\s+Train\s+\|"
                epoch_matches = re.findall(epoch_pattern, data)
                
                time_pattern = r"Epoch time:\s+(\d+\.\d+)"
                time_matches = re.findall(time_pattern, data)

                epoch_numbers = torch.tensor([int(match) for match in epoch_matches], dtype=torch.int32)
                time_values = torch.tensor([float(time) for time in time_matches], dtype=torch.float32)

                epochs_col = epoch_numbers.unsqueeze(1)
                times_col = time_values.unsqueeze(1)
                result_tensor = torch.cat((epochs_col, times_col), dim=1)

                if result_tensor.numel():
                    mean_epoch_time = result_tensor[:,1].mean()
                    max_epoch = result_tensor[:,0].max()
    
                    train_time = max_epoch * mean_epoch_time / 60 / 500

                    if 'Training if finished.' in data:
                        
                        results[ds,seed,e] = train_time
                        


{'N_class': 2,
 'N_feature': 6,
 'N_test': 25,
 'N_train': 70,
 'N_valid': 23,
 'dataname': 'acuteinflammation'}
{'N_class': 3,
 'N_feature': 4,
 'N_test': 79,
 'N_train': 373,
 'N_valid': 124,
 'dataname': 'balancescale'}
{'N_class': 2,
 'N_feature': 9,
 'N_test': 79,
 'N_train': 397,
 'N_valid': 127,
 'dataname': 'breastcancerwisc'}
{'N_class': 3,
 'N_feature': 21,
 'N_test': 79,
 'N_train': 397,
 'N_valid': 127,
 'dataname': 'cardiotocography3clases'}
{'N_class': 3,
 'N_feature': 8,
 'N_test': 79,
 'N_train': 397,
 'N_valid': 127,
 'dataname': 'energyy1'}
{'N_class': 3,
 'N_feature': 8,
 'N_test': 79,
 'N_train': 397,
 'N_valid': 127,
 'dataname': 'energyy2'}
{'N_class': 3,
 'N_feature': 4,
 'N_test': 31,
 'N_train': 88,
 'N_valid': 29,
 'dataname': 'iris'}
{'N_class': 2,
 'N_feature': 5,
 'N_test': 79,
 'N_train': 397,
 'N_valid': 127,
 'dataname': 'mammographic'}
{'N_class': 10,
 'N_feature': 16,
 'N_test': 79,
 'N_train': 397,
 'N_valid': 127,
 'dataname': 'pendigits'}
{'N_class'

In [33]:
results.shape

torch.Size([13, 10, 3])

In [34]:
re = results.mean([1,2])
re.shape

torch.Size([13])

In [35]:
re

tensor([ 9.5397, 21.6374, 11.6093, 19.0386, 15.0579, 10.3290,  7.3446,  5.0140,
        14.1820,  6.4070,  6.4419,  4.5694,  9.6275])